In [1]:
import xml.etree.ElementTree as ET

inputPath   = '../../rawData/arg-microtexts/corpus/en/'
outputPath  = './data/' 


In [2]:
def getEDU2Text(root):
    edu2Text = {}

    for x in root.findall("./edu"):
        edu2Text[x.attrib["id"]] = x.text
    return edu2Text

def getADU2Text(root, edu2Text):
    adu2Text = {}

    for x in root.findall("./edge[@type='seg']"):
        adu2Text[x.attrib['trg']] = edu2Text[x.attrib['src']]
    return adu2Text


In [3]:
def parse(root, adu2Text):

    ATTACK  = "attack"
    SUPPORT = "support"

    relation = {}
    for x in root.findall("./edge[@type='add']"):
        rel = '???'
        print(x.attrib)

In [4]:
# def parseAddEdges(root, adu2Text):

#     ATTACK  = "attack"
#     SUPPORT = "support"

#     relation = {}
#     for x in root.findall("./edge[@type='add']"):
#         rel = '???'

#         print(x.attrib)

#         # if x.attrib["type"] == 'reb':
#         #     rel = 
        

#         addTarget = root.find(".edge[@id='"+x.attrib['trg']+"']")

#         x.set("trg", addTarget.attrib["id"])
#         x.set("type", addTarget.attrib["type"])


        
#         print("target", addTarget.attrib)

#         relation[x.attrib['id']] = x.attrib

In [5]:
def parseAddEdges(root, adu2Text):
    relation = {}
    for x in root.findall("./edge[@type='add']"):
        addTarget = root.find(".edge[@id='"+x.attrib['trg']+"']")
        x.set("trg", addTarget.attrib["trg"])
        x.set("type", addTarget.attrib["type"])
        relation[x.attrib['id']] = x.attrib

In [6]:
def parseUndercutEdges(root, adu2Text):
    relation = {}
    for x in root.findall("./edge[@type='und']"):
        undercutTarget = root.find(".edge[@id='"+x.attrib['trg']+"']")
        x.set("trg", undercutTarget.attrib["src"])
        x.set("type", "attack")
        relation[x.attrib['id']] = x.attrib

In [7]:
def parseOtherEdges(root, adu2Text):
    relation = {}
    for x in root.findall("./edge"):
        if x.attrib["type"] == "reb":
          x.set("type", "attack")
        if x.attrib["type"] == "exa":
          x.set("type", "support")
        if x.attrib["type"] == "sup":
          x.set("type", "support")
        relation[x.attrib['id']] = x.attrib

In [8]:
import os
import networkx as nx
import pandas as pd

def parseXml2Csv(filePath, filename):
  tree = ET.parse(filePath)
  root = tree.getroot()

  edu2Text = getEDU2Text(root)
  adu2Text = getADU2Text(root, edu2Text)

  # for x in root.findall("./edge"):
  #   print(x.attrib)

  # print("---------------------------")

  parseAddEdges(root, adu2Text)
  parseUndercutEdges(root, adu2Text)
  parseOtherEdges(root, adu2Text)

  # for x in root.findall("./edge"):
  #   print(x.attrib)

  G = nx.complete_graph(list(adu2Text.keys()), nx.DiGraph())

  for id in G.nodes:
    G.nodes[id]["text"] = adu2Text[id]

  for id in G.edges:
    G.edges[id]["relationType"] = "neither"
  
  for u,v in G.edges:
    argSrc = adu2Text[u]
    argTrg = adu2Text[v]
    G.edges[(u,v)]["argSrc"] = argSrc
    G.edges[(u,v)]["argTrg"] = argTrg

  for x in root.findall("./edge[@type='attack']"):
    src = x.attrib["src"]
    trg = x.attrib["trg"]
    id = (src, trg)
    G.edges[id]["relationType"]  = "attack"


  for x in root.findall("./edge[@type='support']"):
    src = x.attrib["src"]
    trg = x.attrib["trg"]
    id = (src, trg)
    G.edges[id]["relationType"] = "support"

  edges = []

  for id in G.edges:
    edges.append(G.edges[id])

  df = pd.DataFrame(edges)
  df.to_csv(outputPath+filename+".csv", index=False)



In [9]:
for file in os.listdir(inputPath):
  try:
    if file.split(".")[-1] !=  "xml":
      pass    

    filename    = file.split(".")[0]

    parseXml2Csv(inputPath+filename+".xml", filename)
    
  except:
    print("Error :", filename)

In [10]:

# importing libraries 
import glob 
  
# merging the files 
joined_files = os.path.join(outputPath, "*.csv") 
  
# A list of all joined files is returned 
joined_list = glob.glob(joined_files) 
  
# Finally, the files are joined 
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True) 

df.to_csv(outputPath+"mergedCorpusRelations.csv")